In [12]:
#Importation
import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sklearn as sk
from sklearn import tree
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from textblob import TextBlob
from statistics import mean
from math import ceil
from collections import Counter
warnings.filterwarnings('ignore')

In [25]:
# On ramene les données du prétraitement du dataframe
df = pd.read_pickle("./PICKLE/df_liste_commentaires.pkl")
df

,No DESIGNATION,Reussite,COMMENTAIRES
0,299568158,False,[ obtenu marchand christophe le a le clien...
1,297892193,True,[ point de terminaison optique existante malf...
2,297897767,True,[ point de terminaison optique non existante ...
3,223370510,True,[ point de terminaison optique non existante ...
4,299556603,True,[ point de terminaison optique existante malf...
...,...,...,...
4183,223275005,True,[merci de clrer le rendez vous du client car i...
4184,296611510,True,[ point de terminaison optique existante malf...
4185,223309738,True,[ point de terminaison optique non existante ...
4186,298170718,True,[ point de terminaison optique non existante ...


### Transformer les listes de commentaires en un seul string

In [26]:
def liste_to_string(liste):
    final = ""
    for commentaire in liste :
        final = final + commentaire
    return final

def colonne_liste_to_colonne_string(colonne):
    final_colonne = []
    for i in colonne :
        final_colonne.append(liste_to_string(i))
    return final_colonne

In [27]:
df2 = df.drop(columns=["COMMENTAIRES","No DESIGNATION"],axis=1)
df2["ensemble_commentaires"]=colonne_liste_to_colonne_string(df["COMMENTAIRES"])
df2

,Reussite,ensemble_commentaires
0,False,obtenu marchand christophe le a le client...
1,True,point de terminaison optique existante malfa...
2,True,point de terminaison optique non existante m...
3,True,point de terminaison optique non existante i...
4,True,point de terminaison optique existante malfa...
...,...,...
4183,True,merci de clrer le rendez vous du client car il...
4184,True,point de terminaison optique existante malfa...
4185,True,point de terminaison optique non existante r...
4186,True,point de terminaison optique non existante r...


### Un data set avec le nombre d'apparitions de chaque mot dans des réussites et des échecs

In [30]:
final_stopwords_list = ["Ap.", "Apr.", "GHz", "MHz", "USD", "a", "afin", "ah", "ai", "aie", "aient", "aies", "ait", "alors", "après", "as", "attendu", "au", "au-delà", "au-devant", "aucun", "aucune", "audit", "auprès", "auquel", "aura", "aurai", "auraient", "aurais", "aurait", "auras", "aurez", "auriez", "aurions", "aurons", "auront", "aussi", "autour", "autre", "autres", "autrui", "aux", "auxdites", "auxdits", "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avez", "aviez", "avions", "avons", "ayant", "ayez", "ayons", "b", "bah", "banco", "ben", "bien", "bé", "c", "c'", "c'est", "c'était", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles", "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "celà", "cent", "cents", "cependant", "certain", "certaine", "certaines", "certains", "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "cf.", "cg", "cgr", "chacun", "chacune", "chaque", "chez", "ci", "cinq", "cinquante", "cinquante-cinq", "cinquante-deux", "cinquante-et-un", "cinquante-huit", "cinquante-neuf", "cinquante-quatre", "cinquante-sept", "cinquante-six", "cinquante-trois", "cl", "cm", "cm²", "comme", "contre", "d", "d'", "d'après", "d'un", "d'une", "dans", "de", "depuis", "derrière", "des", "desdites", "desdits", "desquelles", "desquels", "deux", "devant", "devers", "dg", "différentes", "différents", "divers", "diverses", "dix", "dix-huit", "dix-neuf", "dix-sept", "dl", "dm", "donc", "dont", "douze", "du", "dudit", "duquel", "durant", "dès", "déjà", "e", "eh", "elle", "elles", "en", "en-dehors", "encore", "enfin", "entre", "envers", "es", "est", "et", "eu", "eue", "eues", "euh", "eurent", "eus", "eusse", "eussent", "eusses", "eussiez", "eussions", "eut", "eux", "eûmes", "eût", "eûtes", "f", "fait", "fi", "flac", "fors", "furent", "fus", "fusse", "fussent", "fusses", "fussiez", "fussions", "fut", "fûmes", "fût", "fûtes", "g", "gr", "h", "ha", "han", "hein", "hem", "heu", "hg", "hl", "hm", "hm³", "holà", "hop", "hormis", "hors", "huit", "hum", "hé", "i", "ici", "il", "ils", "j", "j'", "j'ai", "j'avais", "j'étais", "jamais", "je", "jusqu'", "jusqu'au", "jusqu'aux", "jusqu'à", "jusque", "k", "kg", "km", "km²", "l", "l'", "l'autre", "l'on", "l'un", "l'une", "la", "laquelle", "le", "lequel", "les", "lesquelles", "lesquels", "leur", "leurs", "lez", "lors", "lorsqu'", "lorsque", "lui", "lès", "m", "m'", "ma", "maint", "mainte", "maintes", "maints", "mais", "malgré", "me", "mes", "mg", "mgr", "mil", "mille", "milliards", "millions", "ml", "mm", "mm²", "moi", "moins", "mon", "moyennant", "mt", "m²", "m³", "même", "mêmes", "n", "n'avait", "n'y", "ne", "neuf", "ni", "non", "nonante", "nonobstant", "nos", "notre", "nous", "nul", "nulle", "nº", "néanmoins", "o", "octante", "oh", "on", "ont", "onze", "or", "ou", "outre", "où", "p", "par", "par-delà", "parbleu", "parce", "parmi", "pas", "passé", "pendant", "personne", "peu", "plus", "plus_d'un", "plus_d'une", "plusieurs", "pour", "pourquoi", "pourtant", "pourvu", "près", "puisqu'", "puisque", "q", "qu", "qu'", "qu'elle", "qu'elles", "qu'il", "qu'ils", "qu'on", "quand", "quant", "quarante", "quarante-cinq", "quarante-deux", "quarante-et-un", "quarante-huit", "quarante-neuf", "quarante-quatre", "quarante-sept", "quarante-six", "quarante-trois", "quatorze", "quatre", "quatre-vingt", "quatre-vingt-cinq", "quatre-vingt-deux", "quatre-vingt-dix", "quatre-vingt-dix-huit", "quatre-vingt-dix-neuf", "quatre-vingt-dix-sept", "quatre-vingt-douze", "quatre-vingt-huit", "quatre-vingt-neuf", "quatre-vingt-onze", "quatre-vingt-quatorze", "quatre-vingt-quatre", "quatre-vingt-quinze", "quatre-vingt-seize", "quatre-vingt-sept", "quatre-vingt-six", "quatre-vingt-treize", "quatre-vingt-trois", "quatre-vingt-un", "quatre-vingt-une", "quatre-vingts", "que", "quel", "quelle", "quelles", "quelqu'", "quelqu'un", "quelqu'une", "quelque", "quelques", "quelques-unes", "quelques-uns", "quels", "qui", "quiconque", "quinze", "quoi", "quoiqu'", "quoique", "r", "revoici", "revoilà", "rien", "s", "s'", "sa", "sans", "sauf", "se", "seize", "selon", "sept", "septante", "sera", "serai", "seraient", "serais", "serait", "seras", "serez", "seriez", "serions", "serons", "seront", "ses", "si", "sinon", "six", "soi", "soient", "sois", "soit", "soixante", "soixante-cinq", "soixante-deux", "soixante-dix", "soixante-dix-huit", "soixante-dix-neuf", "soixante-dix-sept", "soixante-douze", "soixante-et-onze", "soixante-et-un", "soixante-et-une", "soixante-huit", "soixante-neuf", "soixante-quatorze", "soixante-quatre", "soixante-quinze", "soixante-seize", "soixante-sept", "soixante-six", "soixante-treize", "soixante-trois", "sommes", "son", "sont", "sous", "soyez", "soyons", "suis", "suite", "sur", "sus", "t", "t'", "ta", "tacatac", "tandis", "te", "tel", "telle", "telles", "tels", "tes", "toi", "ton", "toujours", "tous", "tout", "toute", "toutefois", "toutes", "treize", "trente", "trente-cinq", "trente-deux", "trente-et-un", "trente-huit", "trente-neuf", "trente-quatre", "trente-sept", "trente-six", "trente-trois", "trois", "très", "tu", "u", "un", "une", "unes", "uns", "v", "vers", "via", "vingt", "vingt-cinq", "vingt-deux", "vingt-huit", "vingt-neuf", "vingt-quatre", "vingt-sept", "vingt-six", "vingt-trois", "vis-à-vis", "voici", "voilà", "vos", "votre", "vous", "w", "x", "y", "z", "zéro", "à", "ç'", "ça", "ès", "étaient", "étais", "était", "étant", "étiez", "étions", "été", "étée", "étées", "étés", "êtes", "être", "ô"]
cv = CountVectorizer(stop_words=final_stopwords_list)
data_cv = cv.fit_transform(df2.ensemble_commentaires)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_dtm.index = df2.index
data_dtm["Reussite"]=df["Reussite"]
data_dtm

,aavnt,ab,abdechafi,abdelali,abdelkadergm,abesoin,abimegm,abonn,abonne,abonnement,...,zibaoui,zm,zombelli,zone,zonerdv,éligible,éligiblegm,éligibleprevoir,étiquetage,Reussite
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4183,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
4184,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
4185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
4186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True


In [17]:
nb_lignes = len(df2)
nb_lignes

4188

In [31]:
nb_reussite = len(df2[df2["Reussite"]==True])
nb_echec = len(df2[df2["Reussite"]==False])
nb_reussite,nb_echec

(3686, 502)

In [50]:
def nombre_apparition(dataframe,label,proportion_reussite,proportion_echec):
    new_dataframe = pd.DataFrame({label:[True,False]})
    for colonne in dataframe.columns:
        if colonne != label:
            somme_true = 0
            somme_false = 0
            for i in range(0,nb_lignes):
                value = dataframe.loc[i,colonne]
                if value != 0:
                    if dataframe.loc[i,label]== True :
                        somme_true += value
                    else:
                        somme_false +=value
            prop_reussite = round(somme_true/proportion_reussite,4)
            prop_echec = round(somme_false/proportion_echec,4)
            if abs(prop_reussite-prop_echec) > 0.3 :
                new_dataframe[colonne]=[prop_reussite,prop_echec]
    return new_dataframe
                        
df3 = nombre_apparition(data_dtm,"Reussite",nb_reussite,nb_echec)
df3

,Reussite,existante,laisse,message,na,ok,optique,plp,point,rdv,terminaison,vocal
0,True,0.8741,0.2035,0.2016,0.1696,1.7784,1.7352,0.3394,1.8288,0.6305,1.6877,0.1959
1,False,0.0000,0.5697,0.5757,0.6474,0.0259,0.5976,0.0239,0.8805,1.1235,0.5677,0.5359


In [51]:
df3.to_pickle("PICKLE/df_proportions_mots.pkl")

In [52]:
def score_type(liste_commentaires,reussite):
    score = 0
    for commentaire in liste_commentaires:
        for colonne in df3.columns :
            if colonne != "Reussite":
                nb_occurence = commentaire.count(colonne)
                score = score + nb_occurence*df3.loc[reussite,colonne]
    return score

In [53]:
def colonne_scores(dataframe,colonne):
    score_reussite = []
    score_echec = []
    for liste_commentaires in df[colonne]:
        score_reussite.append(score_type(liste_commentaires,0))
        score_echec.append(score_type(liste_commentaires,1))
    return(score_reussite,score_echec)    

In [54]:
df["score_reussite"],df["score_echec"]=colonne_scores(df,"COMMENTAIRES")

In [55]:
df

,No DESIGNATION,Reussite,COMMENTAIRES,score_reussite,score_echec
0,299568158,False,[ obtenu marchand christophe le a le clien...,1.2610,2.2470
1,297892193,True,[ point de terminaison optique existante malf...,17.1175,6.5856
2,297897767,True,[ point de terminaison optique non existante ...,26.7736,12.6772
3,223370510,True,[ point de terminaison optique non existante ...,11.6306,2.7709
4,299556603,True,[ point de terminaison optique existante malf...,8.4132,2.7430
...,...,...,...,...,...
4183,223275005,True,[merci de clrer le rendez vous du client car i...,11.4731,6.5099
4184,296611510,True,[ point de terminaison optique existante malf...,13.7052,3.3924
4185,223309738,True,[ point de terminaison optique non existante ...,8.0738,2.7191
4186,298170718,True,[ point de terminaison optique non existante ...,10.4827,3.8685


In [56]:
df.to_pickle("PICKLE/df_score.pkl")